In [1]:
from __future__ import print_function, division

import pandas as pd
import numpy as np

%matplotlib inline

## 读取比分历史数据集，数据更新于2018-01-04。

In [5]:
# EPL files from 1993 to 2017
df = pd.read_csv('../足球数据源/英超历史比分.csv')

# 球队列表
teams = df.HomeTeam.unique()

In [8]:
# 实现统计功能的函数
def get_stats(df, team_name='Arsenal', N=2):
    """
    统计一支球队在英超历史上打出连续N场0比0的次数。特别的，N=1简单地统计了球队打出0比0的次数。
    
    N: int, 连续N场
    """
    # data by team
    df_team = df.loc[(df.HomeTeam==team_name)|(df.AwayTeam==team_name), ]
    
    if N == 1:
        cnt_appear = len(df_team.index)
        cnt_00 = ((df_team.FTHG==0)&(df_team.FTAG==0)).sum()
        ratio_00 = round(((df_team.FTHG==0)&(df_team.FTAG==0)).mean()*100, 1)
    elif N >= 2:
        cnt_appear = 0
        cnt_00 = 0

        count = 0
        for i in df_team.index:
            cnt_appear += 1
            if df_team.loc[i, 'FTHG']==df_team.loc[i, 'FTAG']==0:
                count += 1
            else:
                count = 0
            if count == N:
                #print('连续二场0比0：', df_team.loc[i, ])
                cnt_00 += 1
                count = 0#清零
        ratio_00 = round((cnt_00 / cnt_appear)*100, 2)
    else:
        raise ValueError('N>0')
    return team_name, cnt_appear, cnt_00, ratio_00

## 连续N场0比0比例

In [9]:
# 连续N场
N = 2

# 0比0比例
df_stats = pd.DataFrame({'team':teams, 'cnt_appear':[0]*len(teams), 
                         'cnt_00':[0]*len(teams), 'ratio_00(%)':[0.0]*len(teams)}, 
                        columns=['team', 'cnt_appear', 'cnt_00', 'ratio_00(%)'])

for team_name in teams:
    team_name, cnt_appear, cnt_00, ratio_00 = get_stats(df=df, team_name=team_name, N=N)
    df_stats.loc[df_stats['team']==team_name, 'cnt_appear'] = cnt_appear
    df_stats.loc[df_stats['team']==team_name, 'cnt_00'] = cnt_00
    df_stats.loc[df_stats['team']==team_name, 'ratio_00(%)'] = ratio_00

In [10]:
df_result = df_stats.loc[df_stats['cnt_appear']>400, ].sort_values(by=['ratio_00(%)'])
df_result

,team,cnt_appear,cnt_00,ratio_00(%)
36,West Brom,440,0,0.00
18,Man United,942,1,0.11
23,Leicester,444,1,0.23
4,Man City,752,3,0.40
25,Bolton,494,2,0.40
2,Chelsea,942,4,0.42
16,Blackburn,654,3,0.46
8,Southampton,676,4,0.59
26,Middlesbrough,494,3,0.61
3,Liverpool,942,6,0.64


In [168]:
df_result.to_excel('结果.xlsx', index=False)